In [3]:

!pip3 install diffprivlib
#!pip3 install pandas
#!pip3 install seaborn
import diffprivlib.models as dp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import time

import psutil


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [4]:
start_time = time.time()

url = 'shuttle.trn'
column_names = ['1', ' 2', ' 3', ' 4', '5', '6', '7', '8', '9']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)
url = 'shuttle.tst'
column_names = ['1', ' 2', ' 3', ' 4', '5', '6', '7', '8', '9']

raw_dataset2 = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)


In [5]:
dataset = raw_dataset.copy()
dataset2 = raw_dataset2.copy()

In [6]:
dataset = dataset.dropna()
dataset2 = dataset.dropna()
dataset.head()

,1,2,3,4,5,6,7,8,9
50,21,77,0,28,0,27,48,22,2
55,0,92,0,0,26,36,92,56,4
53,0,82,0,52,-5,29,30,2,1
37,0,76,0,28,18,40,48,8,1
37,0,79,0,34,-26,43,46,2,1


In [7]:
train_dataset = dataset
test_dataset = dataset2


In [8]:
X_train = train_dataset.copy()
X_test = test_dataset.copy()

y_train = X_train.pop('9')
y_test = X_test.pop('9')


In [9]:
from sklearn.linear_model import LogisticRegression as sk_LogisticRegression

logreg = Pipeline([
    ('scaler', Normalizer()), 
    ('clf', sk_LogisticRegression()) 
])

logreg.fit(X_train, y_train)

baseline = logreg.score(X_test, y_test)
print("Non-private baseline R2 score: %.2f" % baseline)


Non-private baseline R2 score: 0.94


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
from diffprivlib.models import LinearRegression
dp_logreg = Pipeline([
    ('scaler', Normalizer()), 
    ('clf', dp.LogisticRegression(epsilon=float("inf"), data_norm=2)) 
])

dp_logreg.fit(X_train, y_train)

baseline2 = dp_logreg.score(X_test, y_test)
print("private baseline R2 score: %.2f" % baseline2)


private baseline R2 score: 0.93


In [11]:
epsilons = np.logspace(-3, 2, 500)
accuracy = []
cpu_Overhead = []
mem_Overhead = []

for epsilon in epsilons:
    dp_logreg.set_params(clf__epsilon=epsilon).fit(X_train, y_train)

    cpu_Overhead.append(psutil.cpu_percent())
    psutil.virtual_memory()
    dict(psutil.virtual_memory()._asdict())
    mem_Overhead.append(psutil.virtual_memory().percent)

    accuracy.append(dp_logreg.score(X_test, y_test))

KeyboardInterrupt: ignored

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))
plt.semilogx(epsilons, accuracy, label="Differentially private linear regression", zorder=10)
plt.semilogx(epsilons, baseline * np.ones_like(epsilons), dashes=[2,2], label="Non-private baseline", zorder=5)
plt.xlabel("epsilon")
plt.ylabel("r-squared score")
plt.ylim(0, 1)
plt.xlim(epsilons[0], epsilons[-1])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.savefig('shuttle.png')

In [ ]:
print(sum((cpu_Overhead))/500)
print(sum((mem_Overhead))/500)

In [ ]:
accuracy_final1 = []
for _ in " "*10: 
  dp_logreg.set_params(clf__epsilon=1, clf__max_iter = 1000).fit(X_train, y_train) 
  accuracy_final1.append(dp_logreg.score(X_test, y_test))


In [ ]:
accuracy_final01 = []
for _ in " "*10: 
  dp_logreg.set_params(clf__epsilon=0.1).fit(X_train, y_train) 
  accuracy_final01.append(dp_logreg.score(X_test, y_test))

In [ ]:
accuracy_final001 = []
for _ in " "*10: 
  dp_logreg.set_params(clf__epsilon=0.01).fit(X_train, y_train) 
  accuracy_final001.append(dp_logreg.score(X_test, y_test))

In [ ]:
accuracy_final = []
accuracy_final1.remove(max(accuracy_final1))
accuracy_final1.remove(min(accuracy_final1))

accuracy_final01.remove(max(accuracy_final01))
accuracy_final01.remove(min(accuracy_final01))

accuracy_final001.remove(max(accuracy_final001))
accuracy_final001.remove(min(accuracy_final001))

accuracy_final.append(sum(accuracy_final1)/10)
accuracy_final.append(sum(accuracy_final01)/10)
accuracy_final.append(sum(accuracy_final001)/10)

In [ ]:
accuracy_final

In [ ]:
accuracy_NP = []
for _ in " "*10: 
  logreg.set_params(clf__solver='lbfgs').fit(X_train, y_train) 
  accuracy_NP.append(logreg.score(X_test, y_test))

In [ ]:
print(sum(accuracy_NP)/10)